In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# -------------------------------
# Simulated EEG data (replace with your real data)
# -------------------------------
# Let's assume:
# - 1000 samples
# - 64 EEG channels
# - 128 time points per sample
# - 4 output classes
num_samples = 1000
num_channels = 64
sequence_length = 128
num_classes = 4

# Simulated EEG data
df = pd.read_csv('../data/preprocessed.csv')
df.head()



,0,-1.6083028,-13.170696,-8.997802,1.347497,5.259585,-1.5213675,2.6515262,4.6510377,3.7816849,...,-8.041514,-6.5636144.1,-4.998779.1,-5.34652.1,-6.389744,-11.171185,-2.3907204,1.0866911,6.9113555,6.1289377
0,0,-2.825397,2.390720,3.955555,-4.216361,2.477656,2.129914,3.955555,-5.172649,5.694261,...,4.129426,7.085226,-2.390720,-2.303785,-3.694750,4.564103,-6.215873,-2.738462,-4.477167,-4.390232
1,0,-15.952625,-3.347008,-15.517949,-9.432479,-16.474237,-6.650549,-14.909402,-12.214408,-10.823443,...,-4.390232,-0.738950,4.477167,12.996825,21.951160,21.081806,21.603418,14.909402,9.345543,6.998291
2,0,-26.558731,-18.560684,-12.040537,-4.216361,-9.432479,1.260562,-2.216850,-3.520879,-5.781197,...,-3.086203,-0.652015,3.607814,15.170208,16.039560,6.128938,3.955555,-1.956044,-2.390720,-4.651038
3,0,-5.607326,6.042003,-4.129426,-5.172649,-7.172161,-11.953602,-7.780708,-0.130403,-2.912332,...,-1.260562,1.608303,4.564103,-4.911844,10.301831,10.127961,8.389256,-2.216850,-4.390232,0.478144
4,0,-10.127961,-13.344566,0.652015,6.215873,1.086691,8.736997,5.172649,-1.695238,1.086691,...,-3.086203,-5.868132,0.130403,5.694261,5.694261,5.520391,2.477656,-10.041026,-8.215385,-3.868620


In [5]:
X = df.drop(columns=[ '0'])
y = df['0']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


X_train = X_train.view(-1, num_channels, sequence_length)
X_test = X_test.view(-1, num_channels, sequence_length)

# -------------------------------
# CNN + LSTM model
# -------------------------------
class CNN_LSTM(nn.Module):
    def __init__(self, num_channels, sequence_length, num_classes):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=128, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        # Calculate new sequence length after pooling
        self.lstm_input_size = 64
        self.seq_len_after_cnn = sequence_length // 4  # two maxpools

        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=64, num_layers=1, batch_first=True)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # x: [batch_size, channels, time_steps]
        x = self.pool1(self.relu1(self.conv1(x)))  # -> [B, 128, L/2]
        x = self.pool2(self.relu2(self.conv2(x)))  # -> [B, 64, L/4]

        # LSTM expects input [batch, seq_len, features]
        x = x.permute(0, 2, 1)  # -> [B, L/4, 64]
        lstm_out, _ = self.lstm(x)  # -> [B, L/4, 64]
        out = lstm_out[:, -1, :]  # take output of last time step

        out = self.fc(out)
        return out

# -------------------------------
# Training
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(num_channels, sequence_length, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
batch_size = 32

for epoch in range(n_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train[indices].to(device), y_train[indices].to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

# -------------------------------
# Evaluation
# -------------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    y_pred_labels = torch.argmax(y_pred, dim=1).cpu().numpy()
    acc = accuracy_score(y_test.numpy(), y_pred_labels)
    print("Test Accuracy:", acc)


RuntimeError: shape '[-1, 64, 128]' is invalid for input of size 711644